# SQL2Circuits development notebook

In [1]:
import os
from data_preparation.queries import QueryGenerator
from data_preparation.prepare import DataPreparation
from data_preparation.database import Database

this_folder = os.path.abspath(os.getcwd())
seed_paths = ["data_preparation//query_seeds//JOB_query_seed_execution_time.json",
              "data_preparation//query_seeds//JOB_query_seed_cardinality.json"]
workload_types = ["E", "C"]

run_id = 1
ty = 1
workload_type = workload_types[ty]
database = Database("IMDB")
generator = QueryGenerator(run_id, database = "IMDB", workload_type = workload_type, query_seed_file_path = seed_paths[ty])

query_file = generator.get_query_file()
#data_preparator = DataPreparation(run_id, query_file, database = database, workload_type = workload_type)


In [2]:
from circuit_preparation.circuits import Circuits
import os

this_folder = os.path.abspath(os.getcwd())
output_folder = this_folder + "//circuit_preparation//data//circuits//" + str(run_id) + "//"
if not os.path.exists(output_folder):
    os.makedirs(output_folder)
    print("The new directory: ", output_folder, " is created.")

circuits = Circuits(run_id, query_file, output_folder, write_cfg_to_file = True, write_pregroup_to_file=True, generate_circuit_png_diagrams = True)
circuits.generate_cfg_diagrams()
circuits.generate_pregroup_diagrams()
circuits.generate_capless_pregroup_diagrams()
circuits.genereate_circuit_diagrams()


c:\Users\valte\AppData\Local\Programs\Python\Python310\lib\site-packages\lambeq\text2diagram\ccg_parser.py:24: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm
